In [18]:
# based on https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_09_2_keras_xfer_cv.ipynb

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import logging, os
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn import metrics
import numpy as np
from tensorflow.keras.models import load_model

BASE_PATH = "../../../../../local_data/practice/tfds/"
DATA_PATH = "../../../../../local_data/tfds/"
OUTPUT_PATH = BASE_PATH+"mnist_transfer/"
os.system("mkdir -p " + OUTPUT_PATH)

train_dataset, validation_dataset = tfds.load(
    'mnist',
    data_dir=DATA_PATH,
    split=["train[:40%]", "train[40%:50%]"],
    as_supervised=True,  # Include labels
)

print(f"Number of training samples: {tf.data.experimental.cardinality(train_dataset)}")
print(f"Number of validation samples: {tf.data.experimental.cardinality(validation_dataset)}")

Number of training samples: 24000
Number of validation samples: 6000


In [19]:
def reshape_image(image, label):
    image = tf.reshape(image, (28, 28, 1))
    return image, label

train_dataset = train_dataset.map(reshape_image)   
def convert_to_rgb(image, label):
    image = tf.image.grayscale_to_rgb(image)
    return image, label

train_dataset = train_dataset.map(convert_to_rgb)   
validation_dataset = validation_dataset.map(convert_to_rgb)   

size = (150, 150)

# train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, size)/ 255.0, y))
# validation_dataset = validation_dataset.map(lambda x, y: (tf.image.resize(x, size)/ 255.0, y))
train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, size), y))
validation_dataset = validation_dataset.map(lambda x, y: (tf.image.resize(x, size), y))

batch_size = 32
train_dataset = train_dataset.cache().batch(batch_size).prefetch(buffer_size=10)
validation_dataset = validation_dataset.cache().batch(batch_size).prefetch(buffer_size=10)

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [20]:
# Load model
filename = "valacc_0.927_epochs_5_date_20250720-202940.h5"
fullpath = f"{OUTPUT_PATH}{filename}"
model = load_model(fullpath)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 sequential_3 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 rescaling_3 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_3  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_3 (Dropout)         (None, 2048)              0         
                                                           

In [21]:
# Make predictions
# probabilities = model.predict(test_dataset.take(10))
probabilities = model.predict(train_dataset)
predict = np.argmax(probabilities, axis=1)
print(predict.shape)
df2 = pd.DataFrame(predict, columns=["pred"])
# print(df2)

allpreds=predict.flatten()
allpnorms = allpreds

# print(predict)

alllabels=np.empty(0)
for images, labels in train_dataset:
    alllabels = np.append(alllabels, labels.numpy().flatten())

score = metrics.accuracy_score(alllabels, allpnorms)
print("Validation accuracy score: {}".format(score))

750/750 [==============================] - 11s 14ms/step
(24000,)
Validation accuracy score: 0.932125


In [22]:
collabels = pd.DataFrame(alllabels, columns=["l"])
colpreds = pd.DataFrame( allpreds, columns=["pred"])
pnorm = pd.DataFrame( allpnorms, columns=["pnorm"])
diff = collabels["l"] - pnorm["pnorm"]

compare = pd.concat([collabels, colpreds,pnorm,diff], axis=1)
compare.columns = ["l", "pred", "pnorm","diff"]
print(compare)

compare.to_csv(OUTPUT_PATH + "pred_test_load.csv", index=False)   

         l  pred  pnorm  diff
0      4.0     4      4   0.0
1      1.0     1      1   0.0
2      0.0     0      0   0.0
3      7.0     7      7   0.0
4      8.0     8      8   0.0
...    ...   ...    ...   ...
23995  9.0     9      9   0.0
23996  3.0     5      5  -2.0
23997  7.0     2      2   5.0
23998  6.0     6      6   0.0
23999  8.0     8      8   0.0

[24000 rows x 4 columns]


In [25]:
# Load the dataset
train_dataset, validation_dataset = tfds.load(
    'mnist',
    data_dir=DATA_PATH,
    split=["train[:40%]", "train[40%:50%]"],
    as_supervised=True,  # Include labels
)
number_of_images=100
allcorrect = (allpnorms == alllabels)

new_df=tfds.as_dataframe(train_dataset.take(number_of_images), metadata)
# new_df
new_df['predictions'] = allpreds[0:number_of_images]
# new_df['pred norm'] = allpnorms[0:number_of_images]
new_df['correct'] = allcorrect[0:number_of_images]
new_df

2025-07-20 21:01:23.237178: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


,image,label,predictions,correct
0,,4,4,True
1,,1,1,True
2,,0,0,True
3,,7,7,True
4,,8,8,True
5,,1,1,True
6,,2,2,True
7,,7,7,True
8,,1,1,True
9,,6,6,True
